In [1]:
import cv2
import numpy as np
import imutils


model_weights = "yolov3.weights"
model_config = "yolov3.cfg"

net = cv2.dnn.readNetFromDarknet(model_config, model_weights)
layer_names = net.getLayerNames()
layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


video = cv2.VideoCapture("video.mp4")


classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

search_query = "person"

timestamps = []

frame_count = 0
while True:
    ret, frame = video.read()

    if not ret:
        break

    frame = imutils.resize(frame, width=400)

    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    outputs = net.forward(layer_names)

    for output in outputs:
        scores = output[5:]
        class_ids = np.argmax(scores, axis=1)
        confidence = scores[np.arange(len(class_ids)), class_ids]

        detections = np.column_stack((confidence, class_ids, output[:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])))
        detections = detections[confidence >= 0.5]

        for detection in detections:
            class_id = int(detection[1])
            confidence = float(detection[0])

            if classes[class_id] == search_query:
                timestamp = (frame_count / video.get(cv2.CAP_PROP_FPS))
                timestamps.append(timestamp)

    frame_count += 1


video.release()

print("Matched timestamps:")
for timestamp in timestamps:
    print(timestamp)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20221220::readNetFromDarknet'
